# PriceAction by ChatGPT - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

/home/zuongthao/PycharmProjects/algo-stock/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [6]:
def prepare_data(data):
    return data

In [23]:
class PriceActionStrategy(Strategy):
    risk_reward = 1.5
    
    def init(self):
        pass
    
    def next(self):
        if len(self.data.Close) < 2:
            return
        
        prev_open, prev_high, prev_low, prev_close = self.data.Open[-2], self.data.High[-2], self.data.Low[-2], self.data.Close[-2]
        curr_open, curr_high, curr_low, curr_close = self.data.Open[-1], self.data.High[-1], self.data.Low[-1], self.data.Close[-1]
        
        body = abs(curr_close - curr_open)
        upper_wick = curr_high - max(curr_open, curr_close)
        lower_wick = min(curr_open, curr_close) - curr_low
        
        if body > upper_wick * 2 and curr_close > curr_open:
            stop_loss = curr_low
            take_profit = curr_close + (curr_close - stop_loss) * self.risk_reward
            self.buy(sl=stop_loss, tp=take_profit)
        
        elif body > lower_wick * 2 and curr_close < curr_open:
            stop_loss = curr_high
            take_profit = curr_close - (stop_loss - curr_close) * self.risk_reward
            self.sell(sl=stop_loss, tp=take_profit)

In [24]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [25]:
prepared_data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [26]:
bt = Backtest(prepared_data, PriceActionStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

ValueError: Long orders require: SL (912.4) < LIMIT (912.77375) < TP (912.6500000000001)

In [ ]:
stats

In [ ]:
stats['_trades']

In [ ]:
# bt.plot()